<a href="https://colab.research.google.com/github/sulthonarifimadudin/uasDL/blob/main/goemotions/goemotions_ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets transformers evaluate accelerate scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("go_emotions")
ds


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [ ]:
ds["train"].column_names, ds["train"][0]


(['text', 'labels', 'id'],
 {'text': "My favourite food is anything I didn't have to cook myself.",
  'labels': [27],
  'id': 'eebbqej'})

In [ ]:
label_names = ds["train"].features["labels"].feature.names
num_labels = len(label_names)
num_labels, label_names[:10]


(28,
 ['admiration',
  'amusement',
  'anger',
  'annoyance',
  'approval',
  'caring',
  'confusion',
  'curiosity',
  'desire',
  'disappointment'])

In [ ]:
ds_small = {
    "train": ds["train"].shuffle(seed=42).select(range(8000)),
    "validation": ds["validation"].shuffle(seed=42).select(range(2000)),
    "test": ds["test"].shuffle(seed=42).select(range(2000)),
}


In [ ]:
from transformers import AutoTokenizer
import torch

model_ckpt = "distilbert-base-uncased"  # cepat buat Colab
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess(example):
    enc = tokenizer(example["text"], truncation=True, max_length=128)
    # multi-hot
    y = [0.0] * num_labels
    for lab in example["labels"]:
        y[lab] = 1.0
    enc["labels"] = torch.tensor(y, dtype=torch.float) # Explicitly set to float
    return enc

tokenized = ds.map(preprocess, remove_columns=ds["train"].column_names)
# Ensure labels are explicitly float32 tensors for the Trainer
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype=torch.float32)
tokenized

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5427
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification

id2label = {i: n for i, n in enumerate(label_names)}
label2id = {n: i for i, n in enumerate(label_names)}

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.problem_type = "multi_label_classification"


In [ ]:
import numpy as np

def preprocess(example):
    enc = tokenizer(example["text"], truncation=True, max_length=128)
    y = np.zeros(num_labels, dtype=np.float32)
    y[example["labels"]] = 1.0
    enc["labels"] = y
    return enc

tokenized = ds.map(preprocess, remove_columns=ds["train"].column_names)


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

def collate_fn(features):
    batch = data_collator(features)
    batch["labels"] = batch["labels"].float()
    return batch


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator_default = DataCollatorWithPadding(tokenizer=tokenizer)

def collate_fn_with_float_labels(features):
    batch = data_collator_default(features)
    batch["labels"] = batch["labels"].float()
    return batch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))          # sigmoid
    preds = (probs >= 0.5).astype(int)
    labels = labels.astype(int)
    return {
        "micro_f1": f1_score(labels, preds, average="micro", zero_division=0),
        "macro_f1": f1_score(labels, preds, average="macro", zero_division=0),
    }

args = TrainingArguments(
    output_dir="goemotions-distilbert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # kalau GPU mendukung
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=collate_fn_with_float_labels, # Use the custom collate_fn
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

/tmp/ipython-input-1265894166.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1
1,0.088900,0.088980,0.533611,0.308554
2,0.082200,0.084082,0.561740,0.379912
3,0.075300,0.083988,0.574276,0.404496


{'eval_loss': 0.0839882642030716,
 'eval_micro_f1': 0.57427586849213,
 'eval_macro_f1': 0.4044962296224236,
 'eval_runtime': 3.1232,
 'eval_samples_per_second': 1737.31,
 'eval_steps_per_second': 108.862,
 'epoch': 3.0}